# B.Tech 2016-20 Computer Science & Engineering (Spring 2018)

In [16]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import requests, os, json
from IPython.display import display, HTML

In [8]:
stres = "https://result-data.firebaseio.com/.json"
stres += "?auth=" + '3LOCmUn3OOVvXtFP48satzXEfJDawgTZVR7i6iac'
course = "https://raw.githubusercontent.com/nightawks/scrapper/master/data/course.json"

data = requests.get(url=stres).json()
cdata = requests.get(url=course).json()

#Filter 16EE data
data = {k:v for (k,v) in data.items() if '16CS' in k[:4]}
    
print ("Total Stuents: %s" % len(data))

Total Stuents: 44


## Date of Birth analysis

In [9]:
if len(data) != 0:
    dob = [v['dob'] for (k, v) in data.items()]

    np_dob = (np.array(dob, dtype='datetime64[s]').view('i8'))
    average_dob = np.mean(np_dob).astype('datetime64[s]').astype(dt.datetime)
    median_dob  = np.median(np_dob).astype('datetime64[s]').astype(dt.datetime)
    minimum_dob = np.min(np_dob).astype('datetime64[s]').astype(dt.datetime)
    maximum_dob = np.max(np_dob).astype('datetime64[s]').astype(dt.datetime)
    print (" Average: %s" % average_dob.strftime("%8B %d, %Y"))
    print ("  Median: %s" % median_dob.strftime("%8B %d, %Y"))
    print ("  Oldest: %s" % minimum_dob.strftime("%8B %d, %Y"))
    print ("Youngest: %s" % maximum_dob.strftime("%8B %d, %Y"))

 Average:  October 11, 1998
  Median: December 27, 1998
  Oldest:    March 14, 1997
Youngest:     June 01, 2000


## Branch Change Analysis

In [10]:
if len(data) != 0:
    original_strengeth = 40
    bc_cgpa = [v['cgpa'][1] for (k, v) in data.items() if int(k[-2:]) > 40]
    bc_cgpa = (np.array(bc_cgpa, dtype='float'))

    bc_count = bc_cgpa.size
    bc_highest = np.max(bc_cgpa)
    bc_lowest = np.min(bc_cgpa)
    bc_average = np.mean(bc_cgpa)
    bc_median = np.median(bc_cgpa)

    print ("Total branch changers: %s" % bc_count)
    print ("\nCGPA (after 2nd sem) for branch change:-")
    print ("Highest: %s" % bc_highest)
    print (" Lowest: %s" % bc_lowest)
    print ("Average: %s" % bc_average)
    print (" Median: %s" % bc_median)

Total branch changers: 5

CGPA (after 2nd sem) for branch change:-
Highest: 9.72
 Lowest: 9.47
Average: 9.610000000000001
 Median: 9.6


## Course wise analysis

In [17]:
if len(data) != 0:
    courses = dict()
    for (k, v) in data.items():
        for (sem, scourses) in v['grades'].items():
            for (course, grade) in scourses.items():
                if course not in courses:
                    courses[course] = list()
                    courses[course].append(grade)
                else:
                    courses[course].append(grade)

    clist = list()

    def other_grade(l):
        return len(l) - l.count('EX') - l.count('A') - l.count('B') - l.count('C') - l.count('D') - l.count('P') - l.count('F') - l.count('WH')

    def analyze_grade(l):
        grade_hash = {'EX': 10, 'A': 9, 'B': 8, 'C': 7, 'D': 6, 'P': 5, 'F': 5}
        hashed_grade = list()
        for grade in l:
            if grade in grade_hash:
                hashed_grade.append(grade_hash[grade])
        hashed_grade = (np.array(hashed_grade, dtype='float'))
        if hashed_grade.size == 0:
            return {'average': 0, 'median': 0}
        return {'average': round(np.mean(hashed_grade), 2), 'median': round(np.median(hashed_grade), 2)}

    grade_labels = 'EX', 'A', 'B', 'C', 'D', 'P', 'F', 'WH', 'Other'
    colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue', 'orange', 'red', 'gray', 'black']

    for course, grades in courses.items():
        course_info = cdata[course]
        clist.append((course, course_info['subnane'], course_info['credit'], len(grades), grades.count('EX'), grades.count('A'), 
                     grades.count('B'), grades.count('C'), grades.count('D'), grades.count('P'),
                     grades.count('F'), grades.count('WH'), other_grade(grades), analyze_grade(grades)['average'], analyze_grade(grades)['median']))
    #     patches, texts = plt.pie([grades.count('EX'), grades.count('A'), 
    #                  grades.count('B'), grades.count('C'), grades.count('D'), grades.count('P'),
    #                  grades.count('F'), grades.count('WH'), other_grade(grades)], labels=grade_labels, colors=colors)
    #     plt.axis('equal')
    #     plt.legend(patches, labels, loc="best")
    #     plt.show()
    def sortByAverage(element):
        return element[-2]

    clist.sort(key=sortByAverage)
    df = pd.DataFrame(data = clist)
    df.columns = ['Subject Code', 'Subject Name', 'Credits', 'Students', 'EX', 'A', 'B', 'C', 'D', 'P', 'F', 'WH', 'Other', 'Average', 'Median']

    display(df)

,Subject Code,Subject Name,Credits,Students,EX,A,B,C,D,P,F,WH,Other,Average,Median
0,CS2S001,Project Seminar,2,44,0,0,0,0,0,0,0,44,0,0.00,0.0
1,ME1L001,Mechanics,4,44,4,4,6,11,10,8,0,0,1,7.00,7.0
2,ME2L501,Elements of Mechanical Engineering,3,1,0,0,0,1,0,0,0,0,0,7.00,7.0
3,HS2L002,Speaking and Presentation,4,22,0,3,3,10,2,1,1,2,0,7.15,7.0
4,CS2L001,Discrete Structures,4,44,1,7,9,14,7,6,0,0,0,7.16,7.0
5,EC2L001,Introduction to Electronics,4,44,5,4,8,10,10,3,4,0,0,7.16,7.0
6,CE2L011,Building materials and Construction,3,6,0,0,1,4,0,0,0,1,0,7.20,7.0
7,PH1L001,Physics,4,44,5,8,7,8,10,6,0,0,0,7.36,7.0
8,EC2L004,Digital Electronics Circuits,4,40,4,5,4,16,7,1,1,2,0,7.39,7.0
9,CS2P002,Data Structure Laboratory,2,44,6,7,8,6,12,5,0,0,0,7.41,7.0
